In [ ]:
#### PART 1

import time, netaddr, json
import pandas as pd

df = pd.read_csv('ip2location.csv')
index = 0
def ipnums(arg):
    arg = int(netaddr.IPAddress(arg))
    return arg

def rangloc(argg):
    global df
    global index
    arg = ipnums(argg)
    timer=time.time()
    while True:
        if arg> int(df.at[index,"high"]):
            index+=1
        elif arg< int(df.at[index,"low"]):
            index-=1
        else:
            timerr=(time.time() - timer)*10
            return {"ip":argg,"int_ip":arg, "region":df.at[index,"region"], "ms":timerr}
    
syntax=str([rangloc("5.5.5.5")]).replace("\'", "\"")
print(json.dumps(json.loads(str(syntax)), indent=3))

In [ ]:
#### PART 2

from zipfile import ZipFile, ZIP_DEFLATED
import re
        
def subs(arg):
    arg = re.sub("[a-z]","0", arg)
    return arg

def writezip(zips,file):
    with ZipFile("server_log.zip") as zf:
        with zf.open("rows.csv") as f:
             dff = pd.read_csv(f)
        dff["ip"] = dff["ip"].apply(subs)
        dff["ip"] = dff["ip"].apply(ipnums)
        dff = dff.sort_values("ip")
        dff["region"] = dff["ip"].apply(lambda x: rangloc(x)["region"])
        
    with ZipFile("server_log2.zip", "w", compression=ZIP_DEFLATED) as zf:
         with zf.open("rows.csv", "w") as f:
                dff.to_csv(f, index=False)

In [ ]:
#### PART 3

from zipfile import ZipFile, ZIP_DEFLATED
import re
import pandas as pd

correctzips=[]
with ZipFile('docs.zip') as zf:
    for info in zf.infolist():
        with zf.open(info.filename) as f:
            content = str(f.read())
        zips=re.findall('(NY|CA|WI|IL)\s([0-9]{5}-[0-9]{4}|[0-9]{5})', content)
        if len(zips)!=0:
            for zi in zips:
                correctzips.append(zi[1])

for z in list(set(correctzips)):
    print(z)

In [ ]:
#### PART 4

import pyproj
from shapely.geometry import box
from zipfile import ZipFile
import geopandas as gpd
import pandas as pd
from geopandas import GeoDataFrame as gdf

#get my df from earlier DFF
with ZipFile("server_log2.zip") as zf:
        with zf.open("rows.csv") as f:
             dff = pd.read_csv(f)
                
#dataframe for graphing DF        
path = gpd.datasets.get_path("naturalearth_lowres")
df = gpd.read_file(path)
    
#make color indexer dict COLORS                
for index in range(len(dff)):
    if dff.at[index,"region"] in list(colors.keys()):
        colors[dff.at[index,"region"]]+=1
    else:
        colors[dff.at[index,"region"]]=1 

def colorassign(name):
    colors={}
    if name in colors:
        instance = int(colors[name])
        if instance>=1:
            if instance<1000:
                return "orange"
        if instance>=1000:
            return "red"
    else:
        return "lightgray"

df["colorname"] = df["name"].apply(colorassign)

crs = pyproj.CRS.from_epsg('3035')
b = box(crs.area_of_use.west, crs.area_of_use.south, crs.area_of_use.east, crs.area_of_use.north)
df["geometry"]=df.intersection(b)
df=df[~df.is_empty]

myplot = df.to_crs(crs).plot(figsize=(8,8), color=df["colorname"], edgecolor="black")

ax = myplot
ax.get_figure().savefig("imagefile", format="svg")